In [2]:
def Transform_Data_first_phase(Data): 
  
  import numpy as np

  START_ID = 8001 
  No_Authors = 0
  for key in Data:
    No_Authors = No_Authors + 1
  Mat_Hypergraph = np.chararray((No_Authors + 1, 30))  
  No_Articles_Author = np.zeros(No_Authors,dtype= int) 
  Authors = {START_ID + i: ('', Mat_Hypergraph[i],0) for i in range(No_Authors)}
  Mat_Hypergraph_Transpose = [] 
  Counter = START_ID  
  Counter_ID = 0 
  Counter_art = 0 
  for key in Data:
    Authors.update({Counter : (key, Data[key])})
    Mat_Hypergraph_Transpose.append(Authors[Counter][1])  
    Counter = Counter + 1
    No_Articles_Author[Counter_ID] = len(Authors[Counter-1][1])
    Counter_ID = Counter_ID + 1
    Counter_art = Counter_art + No_Articles_Author[Counter_ID-1]
  copy_Mat_Hypergraph_Transpose = [] 
  for i in range(No_Authors):
    for j in range(No_Articles_Author[i]):
      copy_Mat_Hypergraph_Transpose.append([Mat_Hypergraph_Transpose[i][j], i])
  hypergraph_matrix = np.zeros((Counter_art,4))
  ARTICLE = ''
  Max_Len_Article = 0
  counter_AoA = 0  
  for i in range(Counter_art):
    ARTICLE = copy_Mat_Hypergraph_Transpose[i][0]
    hypergraph_matrix[i][counter_AoA] = copy_Mat_Hypergraph_Transpose[i][1] + START_ID
    counter_AoA = counter_AoA + 1
    for j in range(i+1, Counter_art):
      if (ARTICLE != -1 and ARTICLE == copy_Mat_Hypergraph_Transpose[j][0]):
        hypergraph_matrix[i][counter_AoA] = copy_Mat_Hypergraph_Transpose[j][1] + START_ID
        counter_AoA = counter_AoA + 1
        copy_Mat_Hypergraph_Transpose[j][0] = -1
    if (Max_Len_Article <= counter_AoA):
      Max_Len_Article = counter_AoA
    counter_AoA = 0
  Data_Matrix = []
  No_Articles = 0
  for i in range(Counter_art):
    if (copy_Mat_Hypergraph_Transpose[i][0] != -1):
      No_Articles = No_Articles + 1
      Data_Matrix.append([hypergraph_matrix[i][k] for k in range (Max_Len_Article)]) 
      Data_Matrix[No_Articles - 1].append(0)
  No_Authors_Article = np.zeros((No_Articles,), dtype=int) 
  counter = 0
  for i in range(No_Articles):
    for j in range(Max_Len_Article + 1): 
      if (Data_Matrix[i][j] != 0):
        counter = counter + 1
    No_Authors_Article[i] = counter
    counter = 0
  largest_article = max(No_Authors_Article)  

  return Data_Matrix
  

In [3]:
def Article_Identifier(input):
  
  import numpy as np
  from collections import defaultdict
  
  Matrix_Data = Transform_Data_first_phase(input)
  HyperEdges = defaultdict(set)   
  for Article_id, article in enumerate(Matrix_Data):
    HyperEdges[frozenset(article)].add(Article_id)  
  Hyperedges_Map = {i: () for i in range(len(HyperEdges))} 
  M = {i: () for i in range(len(HyperEdges))}
  counter = 0 
  for key in enumerate(HyperEdges):
    Hyperedges_Map.update({counter: (key)})
    counter = counter + 1
    M.update({counter - 1: (HyperEdges.get(key)) })

  return (Matrix_Data, len(HyperEdges), Hyperedges_Map)


############################################### 
# This function computes the number of repeatition of each article
def Repeated_Article_Counter(input):

  import numpy as np

  Matrix_Data , No_Articles_Id, map_data = Article_Identifier(input)
  Article_to_Set = {0: set()}
  counter = 0
  for Article_id, article in enumerate(Matrix_Data):
    Article_to_Set.update({counter: (frozenset(article))})
    counter = counter + 1
  Article_Repeated = np.zeros((counter,))
  counter_1 = 0
  for i in range(counter):
    value1 = Article_to_Set[i]
    for j in range(i + 1, counter): 
      value2 = Article_to_Set[j]
      if (value1 == value2 and value1 != (0)): 
        counter_1 = counter_1 + 1
        Article_to_Set[j] = (0)
        Article_Repeated[j] = -1
    if (Article_Repeated[i] != -1):
      Article_Repeated[i] = counter_1 + 1
    counter_1 = 0
  Repeatition_Article = np.zeros((No_Articles_Id, ))
  counter_2 = 0
  for i in range(counter):
    if (Article_Repeated[i] != -1):
      Repeatition_Article[counter_2] = Article_Repeated[i]
      counter_2 = counter_2 + 1

  return (No_Articles_Id, Repeatition_Article, map_data)


def Bipartite_Producer(input):

  import networkx as nx

  Hypergraph = nx.Graph()
  articles_number, repeated_articles, Data_Map = Repeated_Article_Counter(input)
  authors_number = len(input)
  for i in range(articles_number):
    Hypergraph.add_node(i, size = repeated_articles[i]*20, color = (len(Data_Map.get(i)[1]) - 1) * 20)
  for i in range(authors_number):
    Hypergraph.add_node(i + 8001, size = 5, color = 0)
  for i in range(authors_number):
    for j in range(articles_number):
      if ( i + 8001 in Data_Map.get(j)[1] ):
        Hypergraph.add_edge(i + 8001, j, color = (len(Data_Map.get(j)[1]) - 1) * 20, weight = 5)
  node_size = np.zeros((Hypergraph.number_of_nodes(),))
  for i in range(articles_number):
    node_size[i] = repeated_articles[i] * 20
  for i in range(articles_number, Hypergraph.number_of_nodes()):
    node_size[i] = 5
  node_colors = np.zeros((Hypergraph.number_of_nodes() ,)) 
  for i in range(articles_number):
    node_colors[i] = Hypergraph.degree(i) * 20
  for i in range(articles_number, Hypergraph.number_of_nodes() ): 
    node_colors[i] = 0
  edge_colors = [Hypergraph[u][v]['color'] for u,v in Hypergraph.edges()]
  non_pendant_authors = []
  pendant_root = dict()
  for node in Hypergraph.nodes():
    if node > 8000 and Hypergraph.degree(node) == 1:
      for adj in Hypergraph.neighbors(node):
        pendant_root.update({node:adj})  
    elif node > 8000 and Hypergraph.degree(node) != 1:
      non_pendant_authors.append(node) 
  for i in pendant_root:
    Hypergraph.remove_edge(i,pendant_root[i])
    Hypergraph.remove_node(i)
 
  return (Hypergraph, pendant_root, non_pendant_authors, node_size, node_colors, edge_colors)



In [4]:

def Initial_Positioning(articles_No, non_p_authors, radus_inner, radus_outer):

  import math

  position = dict()
  ANGLE1 = 2 * math.pi / len(non_p_authors)
  ANGLE2 = 2 * math.pi / articles_No
  for i in range(len(non_p_authors)):
    dx = ( ( radus_inner ) * math.cos( ANGLE1 * i ) )
    dy = ( ( radus_inner ) * math.sin( ANGLE1 * i ) )
    position.update({non_p_authors[i]:(dx, dy)})
  for i in range(articles_No):
    dx = ( ( radus_outer ) * math.cos( ANGLE2 * i ) )
    dy = ( ( radus_outer ) * math.sin( ANGLE2 * i ) )
    position.update({i:(dx, dy)})
  return position

def Repuls_1(Forces, Positions, np_authors, Repuls_Constant ):

  import math
  for i in range(len(np_authors)-1):
    x_coordinate_node1 = Positions[np_authors[i]][0]
    y_coordinate_node1 = Positions[np_authors[i]][1]
    for j in range(i, len(np_authors)):
      x_coordinate_node2 = Positions[np_authors[j]][0]
      y_coordinate_node2 = Positions[np_authors[j]][1]
      difference_x_coordinates = (x_coordinate_node2-x_coordinate_node1)
      difference_y_coordinates = (y_coordinate_node2-y_coordinate_node1)
      if (difference_x_coordinates != 0 or difference_y_coordinates != 0):
        distanceSquared = (difference_x_coordinates*difference_x_coordinates) + (difference_y_coordinates*difference_y_coordinates)
        distance = math.sqrt(distanceSquared)
        FORCE = Repuls_Constant / (distanceSquared + 1)
        Force_on_x_coordinate = FORCE * difference_x_coordinates / (distance + 1)
        Force_on_y_coordinate = FORCE * difference_y_coordinates / (distance + 1)
        Forces.update({np_authors[i]: (Forces[np_authors[i]][0] - Force_on_x_coordinate, Forces[np_authors[i]][1] - Force_on_y_coordinate )})
        Forces.update({np_authors[j]: (Forces[np_authors[j]][0] + Force_on_x_coordinate, Forces[np_authors[j]][1] + Force_on_y_coordinate )})
  return Forces


def Repuls_2(Forces, Positions, articles_No, np_authors, Repuls_Constant ):

  import math
  for i in range(len(np_authors)):
    x_coordinate_node1 = Positions[np_authors[i]][0]
    y_coordinate_node1 = Positions[np_authors[i]][1]
    for j in range(articles_No):
      x_coordinate_node2 = Positions[j][0]
      y_coordinate_node2 = Positions[j][1]
      difference_x_coordinates = (x_coordinate_node2-x_coordinate_node1)
      difference_y_coordinates = (y_coordinate_node2-y_coordinate_node1)
      if (difference_x_coordinates != 0 or difference_y_coordinates != 0):
        distanceSquared = (difference_x_coordinates*difference_x_coordinates) + (difference_y_coordinates*difference_y_coordinates)
        distance = math.sqrt(distanceSquared)
        FORCE = Repuls_Constant / (distanceSquared + 1)
        Force_on_x_coordinate = FORCE * difference_x_coordinates / (distance + 1)
        Force_on_y_coordinate = FORCE * difference_y_coordinates / (distance + 1)
        Forces.update({np_authors[i]: (Forces[np_authors[i]][0] - Force_on_x_coordinate, Forces[np_authors[i]][1] - Force_on_y_coordinate )})
  return Forces


def Attract(Forces, Positions, Hypergraph, np_authors, Spring_Rest, Attraction_Constant):

  import math
  for i in range(len(np_authors)):  
    x_coordinate_node1 = Positions[np_authors[i]][0]
    y_coordinate_node1 = Positions[np_authors[i]][1] 
    for j in Hypergraph.neighbors(np_authors[i]): 
      x_coordinate_node2 = Positions[j][0]
      y_coordinate_node2 = Positions[j][1]
      difference_x_coordinates = (x_coordinate_node2-x_coordinate_node1)
      difference_y_coordinates = (y_coordinate_node2-y_coordinate_node1)
      if (difference_x_coordinates != 0 or difference_y_coordinates != 0):
        distance = math.sqrt( difference_x_coordinates*difference_x_coordinates + difference_y_coordinates*difference_y_coordinates )
        FORCE = Attraction_Constant * ( distance - (Spring_Rest / Hypergraph.degree(np_authors[i])) )  # FIRST POLICY
#       FORCE = (1 + ((Hypergraph.degree(i)-1)/Hypergraph.degree(i))) * Attraction_Constant * ( distance - Spring_Rest + 1)  # SECOND POLICY
#       FORCE = (Hypergraph.degree(i)) * Attraction_Constant * ( distance - Spring_Rest + 1)  # THIRD POLICY
        Force_on_x_coordinate = FORCE * (difference_x_coordinates + 1) 
        Force_on_y_coordinate = FORCE * (difference_y_coordinates + 1)
        Forces.update({np_authors[i]: (Forces[np_authors[i]][0] + Force_on_x_coordinate, Forces[np_authors[i]][1] + Force_on_y_coordinate )})
  return Forces


def Update_Position(Attraction_forces, Repulsion_forces, Positions, np_authors, Step_Time, Movement_Max):
  
  import math
  for i in range(len(np_authors)):
    x_coordinate_node1 = Positions[np_authors[i]][0]
    y_coordinate_node1 = Positions[np_authors[i]][1]
    difference_x_coordinates = Step_Time * (Attraction_forces[np_authors[i]][0] + Repulsion_forces[np_authors[i]][0])
    difference_y_coordinates = Step_Time * (Attraction_forces[np_authors[i]][1] + Repulsion_forces[np_authors[i]][1])
    diss = (difference_x_coordinates*difference_x_coordinates) + (difference_y_coordinates*difference_y_coordinates)
    if (diss > Movement_Max):
      s = math.sqrt(Movement_Max / diss)
      difference_x_coordinates = difference_x_coordinates * s 
      difference_y_coordinates = difference_y_coordinates * s 
    Positions.update({np_authors[i]: (Positions[np_authors[i]][0] + difference_x_coordinates, Positions[np_authors[i]][1] + difference_y_coordinates )})
  return Positions



def Positioning_pendant_Authors(Positions, Hypergraph, pendant_authors):
  
  import math
  Positioned = set()
  for i in range(len(pendant_authors)):
    for Root in Hypergraph.neighbors(pendant_authors[i]):
      counter = 0
      for j in Hypergraph.neighbors(Root):
        if (Hypergraph.degree(j) == 1 and j not in Positioned):
          Positioned.add(j)
          ANGLE1 = 2 * math.pi / (len(pendant_authors) * 5)
          ANGLE_Root = np.arctan2(Positions[Root][1], Positions[Root][0])
          x_coordinate = 0 + ( 75 * math.cos( (ANGLE1 * counter) + ANGLE_Root ) ) 
          y_coordinate = 0 + ( 75 * math.sin( (ANGLE1 * counter) + ANGLE_Root ) ) 
          counter = counter + 1
          Positions.update({j:(x_coordinate, y_coordinate)})
  return Positions


def Force_directed(hypergraph, articles_No, np_authors, pendant_authors, Starting_positions, Threshold, Number_of_Iteration, Maximum_Movement, Time_step, Spring_Rest_Lentgh, Repulsion_constant, Spring_constant, Distance_limit_Repulsion):

  import numpy as np

  Total_Energy = 0.0  
  Final_Position = {i: (Starting_positions[i][0], Starting_positions[i][1]) for i in hypergraph.nodes()}
  Attraction_force = {np_authors[i]: (0,0) for i in range(len(np_authors))} 
  Repulsion_force = {np_authors[i]: (0,0) for i in range(len(np_authors))} 
  Objective = 1   
  Iteration_Number = 0 
  FLAG_1 = True 
  FLAG_2 = True
  Total_Energy_step_K_1 = 1  
  Total_Energy_step_K = 1  
  Switch = True 

  while (Objective >= Threshold and Iteration_Number < Number_of_Iteration): 
    Iteration_Number = Iteration_Number + 1
    Attraction_force.update({i: (0,0) for i in Attraction_force}) 
    Repulsion_force.update({i: (0,0) for i in Repulsion_force}) 
    Repuls_1(Repulsion_force, Final_Position, np_authors, Repulsion_constant)
    Repuls_2(Repulsion_force, Final_Position, articles_No, np_authors, Repulsion_constant)
    Attract(Attraction_force, Final_Position, hypergraph, np_authors, Spring_Rest_Lentgh, Spring_constant )              
    Update_Position(Attraction_force, Repulsion_force, Final_Position, np_authors, Time_step, Maximum_Movement)
    Sum_forces_on_x_cordinate = 0
    Sum_forces_on_y_cordinate = 0   
    for i in range(len(np_authors)):
      Sum_forces_on_x_cordinate  = np.abs(Attraction_force[np_authors[i]][0] + Repulsion_force[np_authors[i]][0]) + Sum_forces_on_x_cordinate
      Sum_forces_on_y_cordinate  = np.abs(Attraction_force[np_authors[i]][1] + Repulsion_force[np_authors[i]][1]) + Sum_forces_on_y_cordinate
    if (Switch):
      Total_Energy_step_K = Sum_forces_on_x_cordinate + Sum_forces_on_y_cordinate  
      Total_Energy = Total_Energy_step_K
      Switch = False
    else:
      Total_Energy_step_K_1 = Sum_forces_on_x_cordinate + Sum_forces_on_y_cordinate
      Total_Energy = Total_Energy_step_K_1
      Switch = True
    if (FLAG_1):
      Objective = 1
      FLAG_1 = False
    elif (FLAG_2):
      Objective = 1
      FLAG_2 = False
    else:
      Objective = np.abs((Total_Energy_step_K - Total_Energy_step_K_1) / ((Total_Energy_step_K)) )  
 
  return Final_Position



In [18]:

def Crossings(Hypergraph, new_Position, Articles_No): 
  from shapely.geometry import LineString
  import numpy as np
  Crossing_Participation = np.zeros(Articles_No)
  counter = 0
  counter_local = 0
  for i in range(Articles_No):
    point_11_x = new_Position[i][0] 
    point_11_y = new_Position[i][1]
    for j in Hypergraph.neighbors(i):
      point_12_x = new_Position[j][0] 
      point_12_y = new_Position[j][1]
      Line_1 = LineString([(point_11_x, point_11_y), (point_12_x, point_12_y)])
      for k in range(Articles_No):
        if (i != k):
          point_21_x = new_Position[k][0] 
          point_21_y = new_Position[k][1]
          for l in Hypergraph.neighbors(k):
            point_22_x = new_Position[l][0] 
            point_22_y = new_Position[l][1]
            Line_2 = LineString([(point_21_x, point_21_y), (point_22_x, point_22_y)])
            if (Line_1.intersects(Line_2) and j != l):
              counter = counter + 1
              counter_local = counter_local + 1
    Crossing_Participation[i] = counter_local
    counter_local = 0        
  return Crossing_Participation


def Add_and_Position_Pendant_Authors(Hypergraph, pendant_neigbors, No_articles, position):
  
  import math
  import numpy as np

  for i in pendant_neigbors:
    Hypergraph.add_node(i)
    Hypergraph.add_edge(i, pendant_neigbors[i])
  Positioned = set()
  for author in pendant_neigbors:
    if (author not in Positioned):
      counter = 0
      for j in Hypergraph.neighbors(pendant_neigbors[author]):
        if (Hypergraph.degree(j) == 1 and j not in Positioned):
          Positioned.add(j)
          ANGLE1 = 2 * math.pi / (len(pendant_neigbors) * 5)
          ANGLE_Root = np.arctan2(position[pendant_neigbors[author]][1], position[pendant_neigbors[author]][0])
          x_coordinate = 0 + ( 80 * math.cos( (ANGLE1 * counter) + ANGLE_Root ) ) 
          y_coordinate = 0 + ( 80 * math.sin( (ANGLE1 * counter) + ANGLE_Root ) ) 
          counter = counter + 1
          position.update({j:(x_coordinate, y_coordinate)})
  return (Hypergraph, position)

def Swap_Nodes(Node1, Node2, current_position):       
  
  CMx = current_position[Node1][0] 
  CMy = current_position[Node1][1]
  CMx2 = current_position[Node2][0] 
  CMy2 = current_position[Node2][1]
  current_position.update({Node1: (CMx2, CMy2)})
  current_position.update({Node2: (CMx, CMy)})
  return current_position


def check_crossings(Hypergraph, node_1, node_2, crossings_share, if_position):

  from shapely.geometry import LineString
  crossings_participate = {node_1:0, node_2:0}
  old_crossings = crossings_share[node_1] + crossings_share[node_2]
  flag = False
  new_crossings = 0
  vertices = [node_1, node_2]
  counter_local = 0
  for i in range(2):
    point_11_x = if_position[vertices[i]][0] 
    point_11_y = if_position[vertices[i]][1]
    for j in Hypergraph.neighbors(vertices[i]):
      point_12_x = if_position[j][0] 
      point_12_y = if_position[j][1]
      Line_1 = LineString([(point_11_x, point_11_y), (point_12_x, point_12_y)])
      for k in range(len(crossings_share)):
        if (vertices[i] != k):
          point_21_x = if_position[k][0] 
          point_21_y = if_position[k][1]
          for l in Hypergraph.neighbors(k):
            point_22_x = if_position[l][0] 
            point_22_y = if_position[l][1]
            Line_2 = LineString([(point_21_x, point_21_y), (point_22_x, point_22_y)])
            if (Line_1.intersects(Line_2) and j != l):
              new_crossings = new_crossings + 1
              counter_local = counter_local + 1
    crossings_participate[vertices[i]] = counter_local
    counter_local = 0  
  if (new_crossings < old_crossings):
    flag = True
    crossings_share[node_1] = crossings_participate[node_1]
    crossings_share[node_2] = crossings_participate[node_2]
  return (flag, crossings_share)



def Mixed_Discrete_continuous(Hypergraph, crossing_share, np_authors, p_authors, in_position, Threshold, Iterations, Max_Movement, Time_step, Spring_Rest_Lentgh, Repulsion_constant, Spring_constant, Distance_limit_Repulsion):

  import random as rd
  import numpy as np

  n_articles = len(Hypergraph.nodes()) - len(np_authors)
  position = in_position 
  Aforce = {np_authors[i]:(0,0) for i in range(len(np_authors))} 
  Rforce = {np_authors[i]:(0,0) for i in range(len(np_authors))} 
  crossing_before = sum(crossing_share)
  crossing_after = crossing_before + 1
  counter = 1
  pos = dict()
  nodes_crossings = crossing_share
  while (counter < Iterations and crossing_after > crossing_before):
    counter += 1
    check = False
    pos.update({i:(position[i][0], position[i][1]) for i in range(n_articles)})
    pos.update({np_authors[i]:(position[np_authors[i]][0], position[np_authors[i]][1]) for i in range(len(np_authors))})
    ####### choose vertex_1
    First_Set = set()
    Flag_1 = True
    while (Flag_1):
      vertex_1 = rd.randint(0, n_articles - 1)
      if (vertex_1 not in First_Set): 
        First_Set.add(vertex_1)
        Flag_1 = False
    ####### choose vertex_2
    Second_Set = set()
    Flag_2 = True
    while (Flag_2):
      vertex_2 = rd.randint(0, n_articles - 1)
      if (vertex_2 != vertex_1 and vertex_2 not in Second_Set):
        Second_Set.add(vertex_2)
        Flag_2 = False
    pos = Swap_Nodes(vertex_1, vertex_2, pos)
    check = False
    check , crossing = check_crossings(Hypergraph, vertex_1, vertex_2, crossing_share, pos)
    if (check):
      crossing_after = sum(crossing)
      position = Swap_Nodes(vertex_1, vertex_2, position)
      Total_Energy = 0.0
      Objective = 1   
      FLAG_1 = True 
      FLAG_2 = True
      Total_Energy_step_K_1 = 1  
      Total_Energy_step_K = 1  
      Switch = True 
      counter_loop = 0
      while (Objective >= Threshold and counter_loop < Iterations): 

        Aforce.update({i: (0,0) for i in Aforce}) 
        Rforce.update({i: (0,0) for i in Rforce}) 
        Repuls_1(Rforce, position, np_authors, Repulsion_constant)
        Repuls_2(Rforce, position, n_articles, np_authors, Repulsion_constant)
        Attract(Aforce, position, Hypergraph, np_authors, Spring_Rest_Lentgh, Spring_constant )              
        Update_Position(Aforce, Rforce, position, np_authors, Time_step, Max_Movement)
        Sum_forces_on_x_cordinate = 0
        Sum_forces_on_y_cordinate = 0   
        for i in range(len(np_authors)):
          Sum_forces_on_x_cordinate  = np.abs(Aforce[np_authors[i]][0] + Rforce[np_authors[i]][0]) + Sum_forces_on_x_cordinate
          Sum_forces_on_y_cordinate  = np.abs(Aforce[np_authors[i]][1] + Rforce[np_authors[i]][1]) + Sum_forces_on_y_cordinate
        if (Switch):
          Total_Energy_step_K = Sum_forces_on_x_cordinate + Sum_forces_on_y_cordinate  
          Total_Energy = Total_Energy_step_K
          Switch = False
        else:
          Total_Energy_step_K_1 = Sum_forces_on_x_cordinate + Sum_forces_on_y_cordinate
          Total_Energy = Total_Energy_step_K_1
          Switch = True
        if (FLAG_1):
          Objective = 1
          FLAG_1 = False
        elif (FLAG_2):
          Objective = 1
          FLAG_2 = False
        else:
          Objective = np.abs((Total_Energy_step_K - Total_Energy_step_K_1) / ((Total_Energy_step_K)) )  
        counter_loop += 1
  return position
  


In [ ]:

if __name__ == '__main__':
    
  import numpy as np
  import math
  import networkx as nx
  import matplotlib.pyplot as plt
  import re, json, requests

  dataset = requests.get('https://raw.githubusercontent.com/mnafar/Mixed-coordinate-node-link-visualization/main/math_second_comp.json')
  CoAuthorship_Hypergraph = json.loads(dataset.text) 

  threshold = 0.001
  iterations = 200
  max_movement = 18
  time_step = 0.22
  spring_rest_lentgh = 20
  repulsion_constant = 62.5
  spring_constant = 0.01
  distance_limit_repulsion = 20
  Bipartite_Hypergraph, pendants, non_pendants, nodes_sizes, nodes_colors, edge_colors = Bipartite_Producer(CoAuthorship_Hypergraph)
  number_articles = len(Bipartite_Hypergraph.nodes()) - len(non_pendants)
  initial_position = Initial_Positioning(number_articles, non_pendants, 30, 70)
  position_force_directed = Force_directed(Bipartite_Hypergraph, number_articles, non_pendants, pendants, initial_position, threshold, iterations, max_movement, time_step, spring_rest_lentgh, repulsion_constant, spring_constant, distance_limit_repulsion)
  crossing_per_node = Crossings(Bipartite_Hypergraph, position_force_directed, number_articles)
  final_position = position_force_directed
  counter = 0
  while (counter < iterations ):

    position = Mixed_Discrete_continuous(Bipartite_Hypergraph, crossing_per_node, non_pendants, pendants, position_force_directed, threshold, iterations, max_movement, time_step, spring_rest_lentgh, repulsion_constant, spring_constant, distance_limit_repulsion)
    position = Force_directed(Bipartite_Hypergraph, number_articles, non_pendants, pendants, position, threshold, iterations, max_movement, time_step, spring_rest_lentgh, repulsion_constant, spring_constant, distance_limit_repulsion)
    counter += 1

  Bipartite_Hypergraph, position = Add_and_Position_Pendant_Authors(Bipartite_Hypergraph, pendants, number_articles, final_position)

  plt.show()
  nx.draw_networkx_nodes(Bipartite_Hypergraph, position, node_size=nodes_sizes, node_color=nodes_colors)
  nx.draw_networkx_edges(Bipartite_Hypergraph, position, width=1, alpha=0.8, edge_color=edge_colors) 
  plt.axis('on')
  plt.show()

  
